# Solving sudoku, using Genetic algorithm 

## Goal:

in this project, we are about to implement the sudoku problem in genetic description and solving our sudoku using this algorythm.

## Discription:

first we are disscribing our problem in genetic problem-solving description.after implementing the gene and chromosome description, se would generate our first initial population of our chromosomes. for doing so we dont use any hint or information for creating each chromosome and we would generate them completely random. then we would define our fitness function wich is a parameter to generate better generation of each population. then by using fitness function we would create our new generation by using crossover and mutation methods. this methods and their implementation is discussed in following. at the we would use some methods to avoide population to not changing and stuck in the same generation. we would generate population if none of the chromosomes is the goal. to check if we have achieved to our goal statement we check the fitness.

**Gene:** each tile or square is regaregarded as a gene in yhis problem.

**Chromosom:** each sudoku table is wich encludes 9 * 9 genes(tile) is regarded as one chromosome.

so we define a class called chromosome wich includes an array of gene. each gene is each tile in a 9 * 9 sudoku so each chromosome would contain 81 genes. also for each chromosome, we would define a parameter called fitness wich would be calculated in calculate_fitness in the following code.

In [2]:
import numpy
import random
n = 9

class Chromosome:
    
    def __init__(self, genes):
        self.genes = genes
        self.fitness = None
        return
    
    def __lt__(self, other):
        return self.fitness < other.fitness 
    
    def calculate_fitness(self):
        temp_row = numpy.zeros(n)
        temp_column = numpy.zeros(n)
        temp_block = numpy.zeros(n)
        row_sum = 0
        column_sum = 0
        block_sum = 0
        for i in range(0, n):
            temp_row = set(self.genes[i])
            row_sum += len(temp_row)
            temp_row = numpy.zeros(n)

        for i in range(0, n):   
            temp_column = set(self.genes[:, i])
            column_sum += len(temp_column)
            temp_column = numpy.zeros(n)

        for i in range(0, n, 3):
            for j in range(0, n, 3):
                temp_block = self.genes[i:i+3, j:j+3].reshape(1,n)[0]
                block_sum += len(set(temp_block))
                temp_block = numpy.zeros(n)
        self.fitness = column_sum + block_sum + row_sum
        return  

    def mutate(self, mutation_rate, initTable):
        constant_genes = numpy.array(numpy.transpose(numpy.nonzero(initTable)))
        r = random.uniform(0, 1.1)
        while(r > 1): 
            r = random.uniform(0, 1.1)
        success = False
        if (r < mutation_rate):
            while(not success):
                row = random.randint(0, n-1)
                col = random.randint(0, n-1)  
                new_gene = random.randint(1, n)
                if(not [row,col] in constant_genes.tolist()):
                    self.genes[row][col] = new_gene
                    success = True
    
        return success

**__lt__ :**  this function would compare two chromosomes by their fitness parameter. we say chromosome 1 is greater than chromosome 2 if fitness1 is greater and fitness2.


**calculate_fitness:** this function calculate the fitness for each chromosome. for doing so, we count the number of different numbers in each row, column and each 3 * 3 box.
so th max parameter of each of these parameters are 9.(when all volumes in each section are all diffrent).at last we do a summation over all these values. wich is 3 * (9 * 9 ) = 243. so our goal is to achive to this fitness. (remember that we are trying to fill the sudoku over none repeating numbrs from 1 to 9 in each row, col and box so that is the reason we define this function as so.)

**mutate:** this function impliments a mutation method over an chromosome over condition that a random number is in range of mutation range.(this number has been valuated by trying different numbers for mutation range and finding the best numbr. and if the random number was in the mutation range we would choose a random row od the chromosome's gene array and 2 different tiles, and if none of them were constant and we would simply change their positions.

**reason for implementation of mutation in this way:** the reason is when we achive to higher and bettr fitness in our generation, it seems that we have all our numbers in one of the all row or columns or box. so we dont have regenerate any new number and we can easily change to tile's posotions. it would avoid generation to nt generationg new chromosomes.

In [3]:
class Population():

    def __init__(self):
        self.chromosomes  = None
        return

    def create_population(self, numc, initTable):
        row, col = numpy.nonzero(initTable)
        self.chromosomes = []
        for num in range (0,numc):
            genes = numpy.random.randint(1,10,size=(9,9))
            genes[row,col] = initTable[row,col]
            new_chromosome = Chromosome(genes)
            new_chromosome.calculate_fitness()
            self.chromosomes.append(new_chromosome)
        return
    
    def sort(self):
        self.chromosomes.sort(reverse=True, key=self.sort_fitness)
        return

    def sort_fitness(self,x):
        return x.fitness
    
    def creat_fitness_list(self):
        fitness_list = []
        for chrom in self.chromosomes:
            fitness_list.append(chrom.fitness)       
        return fitness_list
        
    def choose_parents(self, fitness_list):       
        c1 = self.chromosomes[random.choices(range(0, len(self.chromosomes)), weights=fitness_list)[0]]
        c2 = self.chromosomes[random.choices(range(0, len(self.chromosomes)), weights=fitness_list)[0]]

        if(c1.fitness > c2.fitness):
            fittest = c1
            weakest = c2
        else:
            fittest = c2
            weakest = c1

        selection_rate = 0.85
        r = random.uniform(0, 1.1)
        while(r > 1):
            r = random.uniform(0, 1.1)
        if(r < selection_rate):
            return fittest
        else:
            return weakest

we wanted to have all our chromosome and in a better word, our population in an array and so we could be able to define some functions like crossover method over our population. so we define another class called population and we put an array of chromosomes in it. now description of each function:

**create_population:** this function is used for generating our initial generation and population. for doing so, we simply generate 1000 (numc) random chromosomes but we should consider that ther are some constent tiles wich we dont want to be replaced by this random numbers.

**reason for the number of population:** more than 1000 population would take a long time to generate and do crossover and mutation methods over this generation also less than 1000 wouldnt sometimes achive to goal state.

**sort and  sort_fitness:** these methods are for sorting our chromosome based on their fitness for speeding up the process.

**creat_fitness_list:** this function is used for returning the list of fitnesses in order to choose chromosomes in a weighted random way for crossover prosses. and each weight is based on fitnesses of each chromosome. The higher fitnes, the better chromosome. and it has more probability to be choosed.

**choose_parents:** by using creat_fitness_list() function, now we are able to choose two parrent from our array of chromosomes to be regarded as the parents of 2 child in next generation. and we choose these cromosomes in a weighted random method eich we use their fitnesses as their weights. also we use an selection rate as one of our **hyper parameters** to pick the parrent with higher fitness with a higher probability for generating the next generation.

In [4]:
    def crossover(parent1, parent2, crossover_rate):
        child1 = Chromosome(numpy.copy(parent1.genes))
        child2 = Chromosome(numpy.copy(parent2.genes))
        r = random.uniform(0, 1.1)
        while(r > 1):
            r = random.uniform(0, 1.1)
        if (r < crossover_rate):  
            bitmask = numpy.random.randint(2, size = n)
            for i in range(0,n):
                if bitmask[i] == 0:
                    child1.genes[i] = parent1.genes[i]
                    child2.genes[i] = parent2.genes[i]
                else :
                    child1.genes[i] = parent2.genes[i]
                    child2.genes[i] = parent1.genes[i]
            child1.calculate_fitness()
            child2.calculate_fitness()
        return child1, child2

**crossover:**  this function would do an crossover method over two chromosomes called parents and we descrodescribe a cross over function over these two and generate two children chromosomes of them. to do so fisr we choose an crossover_rat wich in this problem we set this value to 1 so that it would do the cross over under any condition. then we generate a bitmask array wich contains n random value of 0 or 1. for each index if the bitmax value at this location is zero put the related index row of parent1 1 in child 2 and row[index] cparent 2 in child 2 and if it was 1 do reverse. at last we calculate theire fitnesses and return them. the reason for wich we change their row by row is that we dont want to change the orderd values in a row and we are hoping to achive to goal by using thesame rows but in a diffrdifferent order.

In [5]:
class Sudoku():

    def __init__(self):
        self.initTable = None
        self.population = None
        return
    
    def load(self, path):
        with open(path, "r") as f:
            genes = numpy.loadtxt(f).reshape((n, n)).astype(int)
            self.initTable = Chromosome(genes)
        return
        
    def solve(self):
        numc = 1000
        nume = int(0.05*numc)
        numg = 1000
        numm = 0
        phi = 0
        sigma = 1
        mutation_rate = 0.06
        stale = 0
        
        self.population = Population()
        self.population.create_population(numc, self.initTable.genes) 
        print(self.initTable.genes)
        for generation in range(0, numg):
            #print("Generation : ", generation)
            for c in range(0, numc):
                fitness = self.population.chromosomes[c].fitness
                if(fitness == 234):
                    print("Solution found at generation ", generation, "!")
                    print("fitness: ", self.population.chromosomes[c].fitness)
                    return self.population.chromosomes[c].genes

            # Create the next population.
            next_population = []
            self.population.sort()
            elites = []
            for e in range(0, nume):
                elite = Chromosome(numpy.copy(self.population.chromosomes[e].genes))
                elite.calculate_fitness()
                elites.append(elite)
            #print("fitness: ",elites[0].fitness)
            
             # Create the rest of the chromosomes.
            for count in range(nume, numc, 2):
                fitness_list = self.population.creat_fitness_list()
                parent1 = self.population.choose_parents(fitness_list)
                parent2 = self.population.choose_parents(fitness_list)          
                child1, child2 = crossover(parent1, parent2, crossover_rate=1.0)
                
                old_fitness = child1.fitness
                success = child1.mutate(mutation_rate, self.initTable.genes)
                child1.calculate_fitness()
                if(success):
                    numm += 1
                    if(child1.fitness > old_fitness):
                        phi = phi + 1

                old_fitness = child2.fitness
                success = child2.mutate(mutation_rate, self.initTable.genes)
                child2.calculate_fitness()
                if(success):
                    numm += 1
                    if(child2.fitness > old_fitness):
                        phi = phi + 1
                next_population.append(child1)
                next_population.append(child2)
                
            for e in range(0, nume):
                next_population.append(elites[e])
            self.population  = Population()
            self.population.chromosomes = next_population

            if(numm == 0):
                phi = 0
            else:
                phi = phi / numm            
            if(phi > 0.2):
                sigma = sigma/0.998
            elif(phi < 0.2):
                sigma = sigma*0.998

            mutation_rate = abs(numpy.random.normal(loc=0.0, scale=sigma, size=None))
            numm = 0
            phi = 0
            
            self.population.sort()
            if(self.population.chromosomes[0].fitness != self.population.chromosomes[1].fitness):
                stale = 0
            else:
                stale += 1
                
            if(stale >=100):
                print("The population has gone stale. Re-generating...")
                self.population = Population()
                self.population.create_population(numc, self.initTable.genes) 
                stale = 0
                sigma = 1
                phi = 0
                numm = 0
                mutation_rate = 0.06
        
        print("No solution found.")
        return None               

**sudoku:** sudoku class is defined to be an higher level class to conects all parts together and control them. this class has two parameters. first one is the population wich it save a population of current generation in it.second paramter is the init table wich contains the raw board. so that we could solve the problem based on constent values.

**solve:** solve finally solve the problem using all functions and methods wich we have implemented till now. because it is too long, we would break this code in parts and give a summary of each part.

**part1-hyper parameters:** we define some hyper parameters in order to find the solution in such more efficient way and also avoide loosing good chromosomes after achieving to closer fitnesses to goal fitness and also to avoid generation to get stucked in a same generation.

(**the reason that the algorythm might stuck in a sonstant generation is that the generation might get very simmilar chromosomes in that generation and also each posible children that could be generated from twi chromosomes, allready exists in the generation and also mutaion would decrease the fitness so mush that it would be removed and extincted after some generations.**) 

to avoid getting stuck, we use **stale** parameter wich stores the number of generation that the fitness hasn't changed for more than 100 generation. if so we would regenerate the nation. 

phi is for checking if the gneratin is converged enough or noy and sigma would control the rate of the changes.

mutation,crossover and selection ranges has been discussed previously in the project and as we said they are used for controlling the rate of happening of these methods.then we have phi and sigma wich the idea of this is in refrence[6] wich are implimented to controll the rang of mutaion. as the best fitness in a generation gets higher it would limit the number of mutations to avoid loosing good chromosomes.

**part2-generating the new population:** wich it self contains 3 part **1-** elites wich pass 0.06% of each generation to next generation. **2-** children of crossover method, **3-** mutated childrens. each part is defined priviously in project. the last part we have used a function to generate the mutation rate in order to controll aor generation.

at last if we achiv to fitness 234 it means we have achived to goal and we will print out the tables.

In [9]:
s = Sudoku()
s.load("Test1.txt")
solution = s.solve()
print(solution)

[[8 0 6 0 0 0 1 0 7]
 [0 0 0 6 0 2 0 0 0]
 [0 5 3 0 0 4 8 0 6]
 [7 0 4 8 0 0 6 3 0]
 [0 0 0 0 0 0 0 9 0]
 [1 0 0 5 0 0 4 0 0]
 [0 0 1 2 0 0 7 0 9]
 [2 0 0 0 9 6 0 0 0]
 [0 7 0 0 1 0 0 8 0]]
Solution found at generation  151 !
fitness:  234
[[8 2 6 9 5 3 1 5 7]
 [5 1 7 6 8 2 9 4 3]
 [9 5 3 1 7 4 8 2 6]
 [7 9 4 8 2 1 6 3 8]
 [3 6 5 4 3 7 2 9 1]
 [1 8 2 5 6 9 4 7 5]
 [4 3 1 2 8 5 7 6 9]
 [2 5 8 7 9 6 3 1 4]
 [6 7 9 4 1 3 5 8 2]]


In [6]:
s = Sudoku()
s.load("Test2.txt")
solution = s.solve()
print(solution)

[[0 6 0 2 0 0 0 7 1]
 [4 0 5 0 0 0 0 0 2]
 [3 0 0 0 8 0 6 9 0]
 [2 0 0 9 0 8 7 0 0]
 [0 9 3 0 0 0 8 0 0]
 [0 0 6 0 0 1 0 0 9]
 [0 8 7 0 3 0 0 0 6]
 [6 0 0 0 0 0 5 0 7]
 [0 0 0 0 0 9 0 2 0]]
Solution found at generation  147 !
fitness:  234
[[9 6 8 2 4 5 3 7 1]
 [4 7 5 6 9 3 1 8 2]
 [3 2 1 5 8 7 6 9 4]
 [2 5 4 9 6 8 7 1 3]
 [7 9 3 2 5 4 8 6 5]
 [8 1 6 3 7 1 2 4 9]
 [1 8 7 4 3 2 9 5 6]
 [6 4 9 8 2 6 5 3 7]
 [5 3 2 7 1 9 4 2 8]]


# Result :

as we learned, genetic algorythms can be usefull in problems wich has a large amount of state and checking and searching over all these states might not be possible or it might take a long time or memmory. but as we see it might be stuck in a loop and not change and not find the solution. also because it is based on random acts, sometimes it would find a solution in a very fast time but sometimes it might not even achive to goal. at last it is goog for implimnting some problems such as so.

## References :

1-https://github.com/philongvn99/Sudoku-Genetic-Algorithm/blob/main/sudoku.py
    
2 - https://stackoverflow.com/questions/9755538/how-do-i-create-a-list-of-random-numbers-without-duplicates
    
3 -https://pynative.com/python-weighted-random-choices-with-probability/

4-https://www.w3schools.com/python/trypython.asp?filename=demo_ref_list_sort4

5-https://pynative.com/python-weighted-random-choices-with-probability/